<a href="https://colab.research.google.com/github/SandeeeeeeeeepDey/data-science-11-weeks-progg/blob/main/california_housing_wide_deep_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trying the two architectures of Deep and Wide using PyTorch

In [37]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from sklearn.datasets import fetch_california_housing
import pandas as pd

In [38]:
cali_complete = fetch_california_housing()
data = pd.DataFrame(cali_complete["data"], columns = cali_complete["feature_names"])
target = pd.Series(cali_complete["target"], name = cali_complete["target_names"][0])

In [39]:
train_size = int(len(data)*0.8)
valid_size = round(len(data)*0.1)

In [33]:
X_train, y_train = data[:train_size], target[:train_size]
X_valid, y_valid = data[train_size: train_size+valid_size], target[train_size: train_size+valid_size]
X_test, y_test = data[train_size+valid_size:], target[train_size+valid_size:]

In [34]:
X_valid

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
16512,2.6368,34.0,5.769022,1.051630,1310.0,3.559783,37.72,-121.22
16513,3.6378,17.0,5.432075,1.022642,1802.0,3.400000,37.81,-121.22
16514,2.1186,28.0,4.707581,1.039711,1719.0,3.102888,37.80,-121.22
16515,2.1250,37.0,4.919431,1.052133,521.0,2.469194,37.80,-121.22
16516,2.9950,18.0,4.750988,0.984190,1531.0,3.025692,37.81,-121.21
...,...,...,...,...,...,...,...,...
18571,3.4125,46.0,3.587121,0.973485,1120.0,4.242424,36.92,-121.76
18572,1.6875,52.0,3.089286,1.140306,1102.0,2.811224,36.91,-121.75
18573,1.9609,23.0,3.393617,1.162234,1359.0,3.614362,36.91,-121.76
18574,2.5599,42.0,2.826446,0.966942,2312.0,4.776860,36.91,-121.75


In [142]:
X = X_train.iloc[:, :5], X_train.iloc[:, 2:]

In [143]:
X_train_wide, X_train_deep = X

In [ ]:
X_train_wide

In [71]:
X_train_wide, X_train_deep = X_train.iloc[:, :5], X_train.iloc[:, 2:]
X_valid_wide, X_valid_deep = X_valid.iloc[:, :5], X_valid.iloc[:, 2:]
X_test_wide, X_test_deep = X_test.iloc[:, :5], X_test.iloc[:, 2:]

##Dataset and DataLoader

In [40]:
class CaliforniaHousing(Dataset):
  def __init__(self, data, target):
    self.data = torch.tensor(data.values).float()
    self.target = torch.tensor(target.values).float()
    self.means = torch.tensor(data.values.mean(axis=0)).float()
    self.stds = torch.tensor(data.values.std(axis=0)).float()
    self.data = (self.data - self.means) / self.stds

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
     return self.data[idx], self.target[idx]

In [6]:
train_dataset = CaliforniaHousing(X_train, y_train)
valid_dataset = CaliforniaHousing(X_valid, y_valid)
test_dataset = CaliforniaHousing(X_test, y_test)

In [7]:
batch_size = 32

In [8]:
train_dataloader = DataLoader(train_dataset, batch_size = batch_size)
valid_dataloader = DataLoader(valid_dataset, batch_size = batch_size)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size)

##NeuralNets

In [9]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)


###Deep Wide One

In [47]:
class CaliNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear_relu_concat = nn.Sequential(
        nn.Linear(8, 30),
        nn.ReLU(),
        nn.Linear(30,30),
        nn.ReLU()

    )
    self.op = nn.Linear(38,1)

  def forward(self,x):
    through = self.linear_relu_concat(x)
    # print(x.shape," and ", through.shape)
    concat = torch.cat([x,through], dim = 1)
    output = self.op(concat)
    return output

In [48]:
cali_net = CaliNet().to(device)

####Sanity Check

In [119]:
d,t = next(iter(train_dataloader))

In [120]:
d= d.to(device)
t = t.to(device)

In [121]:
g = cali_net(d)

torch.Size([32, 8])  and  torch.Size([32, 30])


####Continue

In [130]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(cali_net.parameters(), lr = 1e-3)

In [65]:
def train(model, dataloader, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()

  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    pred = model(X)
    loss = loss_fn(pred, y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1)*size
      print(f"loss={loss:>8f}, current={current:>5d}, size={size:>5d}")

In [66]:
def valid(model, dataloader, loss_fn):
  size = len(dataloader.dataset)
  num_batch = len(dataloader)

  model.eval()
  loss, correct = 0,0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)

      loss += loss_fn(pred, y).item()
      correct += torch.sqrt(loss_fn(pred, y))

  loss /= num_batch
  correct /= num_batch

  print(f"acc:{correct:>8f}, loss:{loss:>8f} ")

In [67]:
epoch = 150
for i in range(epoch):
  train(cali_net, train_dataloader, loss_fn, optimizer)
  valid(cali_net, valid_dataloader, loss_fn)
  print(i+1,"th epoch done.")

loss=1.417081, current=16512, size=16512
loss=0.106106, current=1667712, size=16512
loss=1.580430, current=3318912, size=16512
loss=0.119552, current=4970112, size=16512
loss=0.557487, current=6621312, size=16512
loss=1.056013, current=8272512, size=16512
acc:1.485203, loss:2.601893 
1 th epoch done.
loss=1.316974, current=16512, size=16512
loss=0.093328, current=1667712, size=16512
loss=1.577389, current=3318912, size=16512
loss=0.117042, current=4970112, size=16512
loss=0.557173, current=6621312, size=16512
loss=1.074784, current=8272512, size=16512
acc:1.497792, loss:2.641179 
2 th epoch done.
loss=1.307108, current=16512, size=16512
loss=0.087218, current=1667712, size=16512
loss=1.566471, current=3318912, size=16512
loss=0.116274, current=4970112, size=16512
loss=0.555575, current=6621312, size=16512
loss=1.079746, current=8272512, size=16512
acc:1.509650, loss:2.680113 
3 th epoch done.
loss=1.309372, current=16512, size=16512
loss=0.084050, current=1667712, size=16512
loss=1.553

KeyboardInterrupt: 

###Deep Wide Two

In [132]:
class XCaliNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.deep_seq = nn.Sequential(
        nn.Linear(6,30),
        nn.ReLU(),
        nn.Linear(30,30),
        nn.ReLU()
    )
    self.op = nn.Linear(35,1)

  def forward(self, x):
    wide, deep = x
    # print("z")
    # print(deep.shape)
    # print(wide.shape)
    deep = self.deep_seq(deep)
    # print("z")
    concat = torch.cat([deep, wide], axis = 1)
    # print("z")
    output = self.op(concat)
    # print("z")
    return output

In [133]:
xcal = XCaliNet().to(device)

In [134]:
def train_dnw(model, dataloader, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()

  for batch, (X,y) in enumerate(dataloader):
    X_a, X_b = X[:, :5], X[:, 2:]
    X_a, X_b, y = X_a.to(device), X_b.to(device), y.to(device)
    X = X_a, X_b
    pred = model(X)

    loss = loss_fn(pred, y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      current = (1+batch)*size
      print(f"loss:{loss.item():>8f}, current:{current:>5d}")

In [137]:
def valid_dnw(model, dataloader, loss_fn):
  size = len(dataloader.dataset)
  num_batch = len(dataloader)
  model.eval()

  loss, metrics = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X_a, X_b = X[:, :5], X[:, 2:]
      X_a, X_b, y = X_a.to(device), X_b.to(device), y.to(device)
      X = X_a, X_b
      pred = model(X)
      l = loss_fn(pred, y)
      loss += l.item()
      metrics += torch.sqrt(l).item()

  loss /= num_batch
  metrics /= num_batch
  print(f"mse={loss:>8f}, rmse={metrics:>8f}")

In [138]:
for i in range(20):
  train_dnw(xcal, train_dataloader, loss_fn, optimizer)
  valid_dnw(xcal, valid_dataloader, loss_fn)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss:5.677817, current:16512
loss:0.586614, current:1667712
loss:7.678383, current:3318912
loss:0.745359, current:4970112
loss:1.691922, current:6621312
loss:14.158963, current:8272512
mse=9.641601, rmse=2.978421
loss:5.677817, current:16512


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss:0.586614, current:1667712
loss:7.678383, current:3318912
loss:0.745359, current:4970112
loss:1.691922, current:6621312
loss:14.158963, current:8272512
mse=9.641601, rmse=2.978421
loss:5.677817, current:16512
loss:0.586614, current:1667712
loss:7.678383, current:3318912
loss:0.745359, current:4970112
loss:1.691922, current:6621312
loss:14.158963, current:8272512
mse=9.641601, rmse=2.978421
loss:5.677817, current:16512
loss:0.586614, current:1667712
loss:7.678383, current:3318912
loss:0.745359, current:4970112
loss:1.691922, current:6621312
loss:14.158963, current:8272512
mse=9.641601, rmse=2.978421
loss:5.677817, current:16512
loss:0.586614, current:1667712
loss:7.678383, current:3318912
loss:0.745359, current:4970112
loss:1.691922, current:6621312
loss:14.158963, current:8272512
mse=9.641601, rmse=2.978421
loss:5.677817, current:16512
loss:0.586614, current:1667712
loss:7.678383, current:3318912
loss:0.745359, current:4970112
loss:1.691922, current:6621312
loss:14.158963, current: